# Adding a region

The next step is to add a region which we will call `R2`, however, this could equally be called `USA` or `India`. This requires us to undertake a similar process as before of modifying the input simulation data.

This tutorial will follow on from the model in tutorial 1 (including the price change in 2040, but before adding the second agent in tutorial 2). To begin, copy your files from tutorial 1 to a new location.

To add a new region, we begin by modifying the ```settings.toml``` file. We just have to add our new region to the ```regions``` variable, in the 4th line of the file, like so:

```toml
regions = ["R1", "R2"]
```

The process to change the input files, however, takes a bit more time. To achieve this, there must be data for each of the sectors for the new region. This, therefore, requires the modification of every [input file](../inputs/index.rst).

Due to space constraints, we will not show you how to edit all of the files. However, you can access the modified files [here](https://github.com/EnergySystemsModellingLab/MUSE_OS/tree/main/docs/tutorial-code/3-add-region/1-new-region).

Effectively, for this example, we will copy and paste the results for each of the input files from region `R1`, and change the name of the region for the new rows to `R2`. 

We have placed two examples as to how to edit the residential sector below. Again, the edited data are highlighted in **bold**, with the original data in normal text. 

For the sake of brevity, we have omitted the entries for 2040 for the `CommIn.csv` file, however, just make sure to copy and paste the values for 2020 to 2040 here. The full file can be seen [here](https://github.com/EnergySystemsModellingLab/MUSE_OS/blob/main/docs/tutorial-code/3-add-region/1-new-region/technodata/residential/CommIn.csv). 

The following file is the modified ```/technodata/residential/CommIn.csv``` file:

|ProcessName|RegionName|Time|Level|electricity|gas|heat|CO2f|wind|
|-|-|-|-|-|-|-|-|-|
|Unit|-|Year|-|PJ/PJ|PJ/PJ|PJ/PJ|kt/PJ|PJ/PJ|
|gasboiler|R1|2020|fixed|0|1.16|0|0|0|
|heatpump|R1|2020|fixed|0.4|0|0|0|0|
|**gasboiler**|**R2**|**2020**|**fixed**|**0**|**1.16**|**0**|**0**|**0**|
|**heatpump**|**R2**|**2020**|**fixed**|**0.4**|**0**|**0**|**0**|**0**|
|...|...|...|...|...|...|...|...|...|

Whereas the following file is the modified ```/technodata/residential/ExistingCapacity.csv``` file:

|ProcessName|RegionName|Unit|2020|2025|2030|2035|2040|2045|2050|
|-|-|-|-|-|-|-|-|-|-|
|gasboiler|R1|PJ/y|10|5|0|0|0|0|0|
|heatpump|R1|PJ/y|0|0|0|0|0|0|0|
|**gasboiler**|**R2**|**PJ/y**|**10**|**5**|**0**|**0**|**0**|**0**|**0**|
|**heatpump**|**R2**|**PJ/y**|**0**|**0**|**0**|**0**|**0**|**0**|**0**|

We will follow a similar process in the ```/technodata/power/Technodata.csv``` file, copying the lines from `R1` to create `R2`. In this tutorial we will test the scenario where `windturbine` in `R2` has increased capacity limits compared to `R1`, which is highlighted below in bold. The rest of the elements are the same for `R1` as they are for `R2`.

Again, we don't show the entries for 2040, apart from the edited windturbine row, for the sake of brevity.

|ProcessName|RegionName|Time|…|MaxCapacityAddition|MaxCapacityGrowth|TotalCapacityLimit|…|Agent1|
|-|-|-|-|-|-|-|-|-|
|Unit|-|Year|…|PJ|%|PJ|…|New|
|gasCCGT|R1|2020|…|2|1|60|…|1|
|windturbine|R1|2020|…|2|1|60|…|1|
|solarPV|R1|2020|…|2|1|60|…|1|
|gasCCGT|R2|2020|…|2|1|60|…|1|
|windturbine|R2|2020|…|**5**|1|**100**|…|1|
|windturbine|R2|2040|…|**5**|1|**100**|…|1|
|solarPV|R2|2020|…|2|1|60|…|1|
|...|...|...|...|...|...|...|...|...|

Now, go ahead and amend all of the other input files for each of the sectors by copying and pasting the rows from `R1` and replacing the `RegionName` to `R2` for the new rows. You must also modify the `BaseYearExport.csv`, `BaseYearImport.csv` and `Projections.csv` files in the `input` folder in a similar way. In this example we won't include any trade between regions, so all values in `BaseYearExport.csv` and `BaseYearImport.csv` should be left as zero.

All of the edited input files can be seen [here](https://github.com/EnergySystemsModellingLab/MUSE_OS/tree/main/docs/tutorial-code/3-add-region/1-new-region).

Again, we will run the results using the ```python -m pip muse settings.toml``` in the command line, and analyse the data as follows:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
mca_capacity = pd.read_csv(
    "../tutorial-code/3-add-region/1-new-region/Results/MCACapacity.csv"
)
all_years = mca_capacity["year"].unique()

for sector_name, sector_data in mca_capacity.groupby("sector"):
    sector_capacity = (
        sector_data.groupby(["year", "region", "technology"]).sum().reset_index()
    )
    g = sns.FacetGrid(data=sector_capacity, col="region")
    g.map_dataframe(
        lambda data, **kwargs: data.pivot(
            index="year", columns="technology", values="capacity"
        )
        .reindex(all_years)
        .plot(kind="bar", stacked=True, ax=plt.gca())
    )
    g.add_legend()
    g.set_ylabels("Capacity (PJ)")
    g.figure.suptitle(f"{sector_name.capitalize()} Sector:")
    g.figure.subplots_adjust(top=0.8)

Due to the similar natures of the two regions, with the parameters effectively copied and pasted between them, the results are very similar in both `R1` and `R2`. `gassupply1` drops significantly within the gas sector due to the carbon price profile. Which leads to the increasing demand of `heatpump` and falling demand of `gasboiler` in both region `R1` and `R2`. `windturbine` and `solarPV` increase significantly to match this demand of `heatpump`. Region `R2` has a slightly higher share of `windturbine` resulting from the increase in capacity that we introduced.

Have a play around with the various costs data in the technodata files for each of the sectors and technologies to see if different scenarios emerge. Although be careful. In some cases, the constraints on certain technologies will make it impossible for the demand to be met and it will give you an error such as the following:

    message: 'The algorithm terminated successfully and determined that the problem is infeasible.'

To avoid this error message you may have to relax the constraints in the technodata files. For instance, `MaxCapacityGrowth`.

## Next steps

In the next section we modify the `settings.toml` file to change the timeslicing arrangements as well as project until 2040, instead of 2050, in two benchmark year time steps.